In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
import pandas as pd
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool

In [15]:
df = pd.read_csv("all_match_data.csv")

In [18]:
df.head()

,datetime,gametype,map_info,result,players
0,2024-05-10 03:32,rating,"{'mapId': '103', 'name': '브리스톨'}","[{'result': 'win', 'players': ['e5ddaa0d96061c...",[{'playerId': 'e5ddaa0d96061c0d5346976a3b24c45...
1,2024-05-10 02:02,rating,"{'mapId': '101', 'name': '리버포드'}","[{'result': 'lose', 'players': ['00ddfcf8c7920...",[{'playerId': '00ddfcf8c79209bd8afe8ec7e49a803...
2,2024-05-10 01:30,rating,"{'mapId': '101', 'name': '리버포드'}","[{'result': 'lose', 'players': ['b2612be939898...",[{'playerId': 'b2612be939898da38f08143a09c9741...
3,2024-05-10 00:46,rating,"{'mapId': '106', 'name': '리버포드 앳 던'}","[{'result': 'lose', 'players': ['58eeac8a6f03e...",[{'playerId': '58eeac8a6f03e5d2596abedd0814192...
4,2024-05-10 00:23,rating,"{'mapId': '104', 'name': '스프링필드'}","[{'result': 'lose', 'players': ['e5ddaa0d96061...",[{'playerId': 'e5ddaa0d96061c0d5346976a3b24c45...


In [19]:
df.loc[0, 'players']

"[{'playerId': 'e5ddaa0d96061c0d5346976a3b24c45b', 'nickname': '날비', 'map': {'mapId': '103', 'name': '브리스톨'}, 'playInfo': {'random': False, 'partyUserCount': 0, 'partyId': 'b762303013093c43599c55f64fdcff53', 'playTypeName': '정상', 'characterId': '149913567cfcc642a07e46ad41049da6', 'characterName': '제키엘', 'level': 39, 'killCount': 2, 'deathCount': 4, 'assistCount': 9, 'attackPoint': 9838, 'damagePoint': 36439, 'battlePoint': 259, 'sightPoint': 462, 'towerAttackPoint': 2395, 'backAttackCount': 38, 'comboCount': 47, 'spellCount': 0, 'healAmount': 0, 'sentinelKillCount': 12, 'demolisherKillCount': 2, 'trooperKillCount': 0, 'guardianKillCount': 0, 'guardTowerKillCount': 0, 'getCoin': 12795, 'spendCoin': 12250, 'spendConsumablesCoin': 3600, 'playTime': 1058, 'responseTime': 67, 'minLifeTime': 91, 'maxLifeTime': 515}, 'position': {'name': '탱커', 'explain': '체력 +7%, 회피 +5%', 'attribute': [{'level': 1, 'id': '678bca255e575ae96aceefacaa7aee4e', 'name': '최후의 저항'}, {'level': 2, 'id': '711f424bc5801d

In [10]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.callbacks.base import BaseCallbackHandler


class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs):
        print(token, end="", flush=True)


# # 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(
        temperature=0,
        model_name="gpt-3.5-turbo-0613",
        streaming=True,
        callbacks=[StreamCallback()],
    ),  # 모델 정의
    df,  # 데이터프레임
    verbose=True,  # 추론과정 출력
    # AgentType.ZERO_SHOT_REACT_DESCRIPTION
    agent_type=AgentType.OPENAI_FUNCTIONS,
)

In [11]:
agent.invoke({"input": "데이터의 행과 열의 갯수는 어떻게 돼?"})



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'Request too large for gpt-3.5-turbo-0613 in organization org-4YiCQLTL7cmBEh5bsflZ4gvv on tokens per min (TPM): Limit 60000, Requested 86739. The input or output tokens must be reduced in order to run successfully. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}